## Processing of genetech data. 
Run th file in BCB cluster


In [ ]:
library(data.table)

genetech_all = fread("/liulab/asahu/data/ssgsea/xiaoman/biomarker_Genetech.txt")
pathway_order = fread("/liulab/asahu/data/ssgsea/xiaoman/ssgsea.order_tcga.txt")
head(colnames(genetech_all))
genetech_all[1:10, 1:10, with=F]
genetech_all[1, seq( ncol(genetech_all)-40,  ncol(genetech_all)), with=F]
# ssgsea.inx 
# load("/liulab/asahu/data/ssgsea/xiaoman/ssgsea.order.list.RData")
ssgsea.inx = match(pathway_order$pathway, colnames(genetech_all))
stopifnot(sum(is.na(ssgsea.inx)) == 0) 
genetech_ssgsea_mat = matrix(genetech_all[,ssgsea.inx, with=F])
empty_slots = grepl(tcga.phenotypes, pattern="empty")
col.genetech = colnames(genetech_all)
col.genetech = gsub(col.genetech, pattern="\\.$", replacement= "")
col.genetech = gsub(col.genetech, pattern="\\.\\.", replacement= ".")
col.genetech = gsub(col.genetech, pattern="\\.", replacement= "_")
col.genetech = gsub(col.genetech, pattern="Tregs", replacement= "(Tregs)")
col.genetech = gsub(col.genetech, pattern="TAM_M2", replacement= "M2")

tcga.phenotypes_ne = tcga.phenotypes[!empty_slots]
genentech.inx = match(tcga.phenotypes_ne, col.genetech)
genentech.without.survival = genentech.inx[-(1:2)]
genetech_phenotype =  genetech_all[,genentech.without.survival, with=F]

ext.mat = matrix(NA, nrow=nrow(genetech_phenotype), ncol=10)
colnames(ext.mat) = c("Responder", "Nonresponder", paste("empty", 3:10))
genetech_phenotype_na = cbind(NA, NA, genetech_phenotype, ext.mat)
genetech_phenotype_na$Responder = ifelse(genetech_all$Responder, 1, 0)
genetech_phenotype_na$Nonresponder = ifelse(genetech_all$Responder, 0, 1)

# genetech_ssgsea$No.benefits
# tcga_ssgsea_mat = t(as.matrix(tcga_ssgsea[,-V1, with=F]))
tcga_ssgsea_mat= t(as.matrix(tcga_ssgsea[,seq(2,ncol(tcga_ssgsea)),with=F]))
colnames(tcga_ssgsea_mat) = tcga_ssgsea$V1
identical(tcga_ssgsea$V1, pathway_order$pathway)
tcga_ssgsea_mat = tcga_ssgsea_mat[,pathway_order$order]
patient.name = substr(rownames(tcga_ssgsea_mat), 1,12)
# phenotype data
tcga_phenotype = fread("/liulab/asahu/data/ssgsea/xiaoman/biomarker_TCGA_type.txt")
tcga_phenotype = tcga_phenotype[!is.na(PDCD1)]
identical(patient.name, tcga_phenotype$bcr_patient_barcode)

tcga_ssgsea_sel = tcga_ssgsea_mat[patient.name %in% tcga_phenotype$bcr_patient_barcode, ] 
patient.name.sel = substr(rownames(tcga_ssgsea_sel), 1,12)
phenotype_sel = tcga_phenotype[match(patient.name.sel, tcga_phenotype$bcr_patient_barcode)]

ext.mat = matrix(NA, nrow=nrow(phenotype.mat), ncol=10)


# introduce 10 NA 

phenotype.dt = phenotype_sel[,setdiff(seq(ncol(phenotype_sel)), c(1,4,5,29,30,39)),with=F]
phenotype.mat = as.matrix(phenotype.dt)
phenotype.mat = apply(phenotype.mat, c(1,2), as.numeric)
ext.mat = matrix(NA, nrow=nrow(phenotype.mat), ncol=10)
colnames(ext.mat) = paste("empty", 1:10)

load(file="/liulab/asahu/data/ssgsea/xiaoman/processed/tcga_phenotypes.RData")
# phenotypes.tcga = fread(file="/liulab/asahu/data/ssgsea/xiaoman/processed/tcga_phenotypes.txt")
phenotpyes = c( gsub(colnames(phenotype.dt), pattern=" ", replacement="_"),colnames(ext.mat) )
phenotype.ext.mat =  cbind(phenotype.mat, ext.mat)

colnames(phenotype.ext.mat) = phenotpyes




# xx = data.table(tissue=substr(rownames(tcga_ssgsea_mat), 1,12), sample=substr(rownames(tcga_ssgsea_mat), 14,15))
dir.create("/liulab/asahu/data/ssgsea/xiaoman/processed")
write.table(file="/liulab/asahu/data/ssgsea/xiaoman/processed/tcga_ssgsea.txt",x = tcga_ssgsea_sel,
            row.names = F, col.names =T,  sep="\t", quote=F )
write.table(file="/liulab/asahu/data/ssgsea/xiaoman/processed/tcga_phenotype.txt",x = phenotype.ext.mat,
            row.names = F, col.names =T,  sep="\t", quote=F )

In [2]:
# normalize survival time with cancer type 
# /n/groups/liu/xiaoman/biomarker_TCGA_type.txt

normalize.std = function(tt){
    (tt - min(tt, na.rm=T))/(max(tt,na.rm=T) - min(tt, na.rm=T))
}
cancer.types = unique(phenotype_sel$cancertype)
survival_time = phenotype_sel$survive
for(ii in cancer.types){
    type.inx = which(phenotype_sel$cancertype ==ii)
    survival_time[type.inx] = normalize.std(survival_time[type.inx])
}
phenotype.ext.mat[,1] = survival_time


In [ ]:
rand_inx = sample(nrow(tcga_ssgsea_sel))
tcga_ssgsea_sel_shuffle = tcga_ssgsea_sel[rand_inx,:]
phenotype.ext.mat_shuffle = phenotype.ext.mat[rand_inx,:]

write.table(file="/liulab/asahu/data/ssgsea/xiaoman/processed/tcga_ssgsea_train.txt",x = tcga_ssgsea_sel_shuffle[1:5000,],
            row.names = F, col.names =T,  sep="\t", quote=F )
write.table(file="/liulab/asahu/data/ssgsea/xiaoman/processed/tcga_phenotype_train.txt",x = phenotype.ext.mat_shuffle[1:5000,],
            row.names = F, col.names =T,  sep="\t", quote=F )

write.table(file="/liulab/asahu/data/ssgsea/xiaoman/processed/tcga_ssgsea_test.txt",x = tcga_ssgsea_sel_shuffle[5001:5500,],
            row.names = F, col.names =T,  sep="\t", quote=F )
write.table(file="/liulab/asahu/data/ssgsea/xiaoman/processed/tcga_phenotype_test.txt",x = phenotype.ext.mat_shuffle[5001:5500,],
            row.names = F, col.names =T,  sep="\t", quote=F )

write.table(file="/liulab/asahu/data/ssgsea/xiaoman/processed/tcga_ssgsea_val.txt",x = tcga_ssgsea_sel_shuffle[5500:nrow(tcga_ssgsea_sel_shuffle),],
            row.names = F, col.names =T,  sep="\t", quote=F )
write.table(file="/liulab/asahu/data/ssgsea/xiaoman/processed/tcga_phenotype_val.txt",x = phenotype.ext.mat_shuffle[5500:nrow(tcga_ssgsea_sel_shuffle),],
            row.names = F, col.names =T,  sep="\t", quote=F )

system("cp -rf /liulab/asahu/data/ssgsea/xiaoman/processed/tcga_*.txt ~/local_data/processed/datasets")

In [3]:
dim(ssgsea.dt)
write.table(file="/home/as892/project/icb/risk/models/ssgsea/ssgsea.txt",  x = ssgsea.dt[1:10,],  sep="\t", quote=F, row.names=F, col.names=T )

[1] 10912  4872

In [11]:
system("mkdir /home/as892/project/icb/risk/models/tcga")
system("mv /home/as892/survData.Rdata /home/as892/project/icb/risk/models/tcga/.")

In [5]:
# load("/home/as892/project/icb/risk/models/tcga/survData.Rdata")
# load("/home/as892/project/icb/risk/models/tcga/pancancer_clinical_tcgabiolinks.RData")
Clinicaldata=read.delim('~/data/tcga/clinical_PANCAN_patient_with_followup.tsv',stringsAsFactors = F)
#extract ID,survive info and censor info
Clinicaldata_sub=Clinicaldata[,c(2,5,7,8)]
#assemble data
Clinicaldata_sub[Clinicaldata_sub$vital_status=='Dead','vital_status']=1
Clinicaldata_sub[Clinicaldata_sub$vital_status=='Alive','vital_status']=0
Clinicaldata_sub$survive=NA
Clinicaldata_sub[Clinicaldata_sub$vital_status==1,'survive']<-
  Clinicaldata_sub[Clinicaldata_sub$vital_status==1,'days_to_death']
Clinicaldata_sub[Clinicaldata_sub$vital_status==0,'survive']<-
  Clinicaldata_sub[Clinicaldata_sub$vital_status==0,'days_to_last_followup']
Clinicaldata_sub$bcr_patient_barcode=gsub(pattern = '-',replacement = '.',
                                          Clinicaldata_sub$bcr_patient_barcode)
Clinicaldata_sub=Clinicaldata_sub[,c(1,5,2)]
Clinicaldata_sub$survive=as.numeric(Clinicaldata_sub$survive)
Clinicaldata_sub$vital_status=as.numeric(Clinicaldata_sub$vital_status)
Clinicaldata_sub = Clinicaldata_sub[!(is.na(Clinicaldata_sub$survive) | is.na(Clinicaldata_sub$vital_status)),]

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

In [8]:
dim(Clinicaldata_sub)
dim(ssgsea.dt)

[1] 10956     3

[1] 10912  4872

### 1 is death and 0 is censor  

In [4]:
head(rownames(ssgsea.dt))

[1] "TCGA.OR.A5J1.01A" "TCGA.OR.A5J2.01A" "TCGA.OR.A5J3.01A" "TCGA.OR.A5J5.01A"
[5] "TCGA.OR.A5J6.01A" "TCGA.OR.A5J7.01A"

In [11]:
ssgsea.samp = substr(rownames(ssgsea.dt),1,12) 
common.samp = intersect(ssgsea.samp, Clinicaldata_sub$bcr_patient_barcode)
length(common.samp)
ssgsea.matched = ssgsea.dt[match(common.samp, ssgsea.samp), ]
survival.matched = Clinicaldata_sub[match(common.samp, Clinicaldata_sub$bcr_patient_barcode), ]

[1] 10049

In [14]:
write.table(file="/home/as892/project/icb/data/tcga_ssgsea.txt",x = ssgsea.matched,
            row.names = F, col.names =T,  sep="\t", quote=F )
write.table(file="/home/as892/project/icb/data/tcga_survival.txt",x = survival.matched[,2:3],
            row.names = F, col.names =T,  sep="\t", quote=F )